# Finding Optimal Locations for New Stores using HERE

This notebook is an example of how **Decision Optimization** can help to prescribe decisions for a complex constrained problem.

It is an updated notebook of <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/aceccfd155454fd9741852e12e9cce4e" target="_blank" rel="noopener noreferrer">Finding Optimal Locations for New Stores</a> with the inclusion of using HERE's APIs, to better show the distances and travels.

When you finish this notebook, you'll have a foundational knowledge of _Prescriptive Analytics_.

>This notebook requires the Commercial Edition of CPLEX engines, which is included in the Default Python 3.7 XS + DO in Watson Studio.

Table of contents:

-  [Describe the business problem](#Describe-the-business-problem)
*  [How decision optimization (prescriptive analytics) can help](#How--decision-optimization-can-help)
*  [Use decision optimization](#Use-decision-optimization)
    *  [Step 1: Download the library](#Step-1:-Download-the-library)
    *  [Step 2: Set up the engines](#Step-2:-Set-up-the-prescriptive-engine)
    -  [Step 3: Model the data](#Step-3:-Model-the-data)
    *  [Step 4: Prepare the data](#Step-4:-Prepare-the-data)
    -  [Step 5: Set up the prescriptive model](#Step-5:-Set-up-the-prescriptive-model)
        * [Define the decision variables](#Define-the-decision-variables)
        * [Express the business constraints](#Express-the-business-constraints)
        * [Express the objective](#Express-the-objective)
        * [Solve with the Decision Optimization solve service](#Solve-with-the-Decision-Optimization-solve-service)
    *  [Step 6: Investigate the solution and run an example analysis](#Step-6:-Investigate-the-solution-and-then-run-an-example-analysis)
*  [Summary](#Summary)

****

## Describe the business problem

* A fictional Coffee Company plans to open N shops in the near future and needs to determine where they should be located knowing the following:
    * Most of the customers of this coffee brewer enjoy reading and borrowing books, so the goal is to locate those shops in such a way that all the city public libraries are within minimal walking distance.
* We use <a href="https://data.cityofchicago.org" target="_blank" rel="noopener noreferrer">Chicago open data</a> for this example.
* We implement a <a href="https://en.wikipedia.org/wiki/K-medians_clustering" target="_blank" rel="noopener noreferrer">K-Median model</a> to get the optimal location of our future shops.

## How  decision optimization can help

* Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes.  It takes into account specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and have greater control of business outcomes.  

* Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes.  

* Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle that future situation. Organizations that can act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.  
<br/>

With prescriptive analytics, you can: 

* Automate the complex decisions and trade-offs to better manage your limited resources.
* Take advantage of a future opportunity or mitigate a future risk.
* Proactively update recommendations based on changing events.
* Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.

## HERE integration

To integrate with HERE Location Services an API Key is required.
1. <a href="https://developer.here.com/ref/IBM_starterkit_Watson?create=Freemium-Basic" target="_blank">Register for a free HERE account</a>
1. <a href="https://developer.here.com/documentation/authentication/dev_guide/topics/api-key-credentials.html" target="_blank">Generate an API Key</a>

Check out the APIs here:
- <a href="https://developer.here.com/documentation/map-tile/dev_guide/topics/introduction.html" target="_blank">Map Tile REST API</a>
- <a href="https://developer.here.com/documentation/routing/dev_guide/topics/resources.html" target="_blank">Routing REST API v7</a>

<br>

Set `HERE_APIKEY` below with the API Key generated in the HERE Developer Portal


In [ ]:
# The code was removed by Watson Studio for sharing.
HERE_APIKEY = ""

## Use decision optimization

### Step 1: Import the docplex package 

This package is presintalled on Watson Studio.

In [ ]:
import sys
import docplex.mp

<i>Note that the more global package docplex contains another subpackage docplex.cp that is dedicated to Constraint Programming, another branch of optimization.</i>

### Step 2: Model the data

- The data for this problem is quite simple: it is composed of the list of public libraries and their geographical locations.
- The data is acquired from <a href="https://data.cityofchicago.org" target="_blank" rel="noopener noreferrer">Chicago open data</a> as a JSON file, which is in the following format:
<code>
data" : [ [ 1, "13BFA4C7-78CE-4D83-B53D-B57C60B701CF", 1, 1441918880, "885709", 1441918880, "885709", null, "Albany Park", "M, W: 10AM-6PM;  TU, TH: 12PM-8PM; F, SA: 9AM-5PM; SU: Closed", "Yes", "Yes ", "3401 W. Foster Avenue", "CHICAGO", "IL", "60625", "(773) 539-5450", [ "http://www.chipublib.org/locations/1/", null ], [ null, "41.975456", "-87.71409", null, false ] ]
</code>
This code snippet represents library "**3401 W. Foster Avenue**" located at **41.975456, -87.71409**


### Step 3: Prepare the data
We need to collect the list of public libraries locations and keep their names, latitudes, and longitudes.

In [ ]:
# Store longitude, latitude and street crossing name of each public library location.
class XPoint(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.waypoint = '{},{}'.format(y, x)
    def __str__(self):
        return "P(%g_%g)" % (self.x, self.y)

class NamedPoint(XPoint):
    def __init__(self, name, x, y):
        XPoint.__init__(self, x, y)
        self.name = name
    def __str__(self):
        return self.name

#### Declare the list of libraries
Parse the JSON file to get the list of libraries and store them as Python elements.

In [ ]:
def build_libraries_from_url(url, name_pos, lat_long_pos):
    import requests
    import json

    r = requests.get(url)
    myjson = json.loads(r.text, parse_constant='utf-8')
    myjson = myjson['data']

    libraries = []
    k = 1
    for location in myjson:
        uname = location[name_pos]
        try:
            latitude = float(location[lat_long_pos][1])
            longitude = float(location[lat_long_pos][2])
        except TypeError:
            latitude = longitude = None
        try:
            name = str(uname)
        except:
            name = "???"
        name = "P_%s_%d" % (name, k)
        if latitude and longitude:
            cp = NamedPoint(name, longitude, latitude)
            libraries.append(cp)
            k += 1
    return libraries

In [ ]:
libraries = build_libraries_from_url('https://data.cityofchicago.org/api/views/x8fc-8rcq/rows.json?accessType=DOWNLOAD',
                                   name_pos=10,
                                   lat_long_pos=16)

In [ ]:
print("There are %d public libraries in Chicago" % (len(libraries)))

#### Define number of shops to open
Create a constant that indicates how many coffee shops we would like to open.

In [ ]:
nb_shops = 5
print("We would like to open %d coffee shops" % nb_shops)

#### Validate the data by displaying them
We will use the <a href="https://folium.readthedocs.org/en/latest/quickstart.html#getting-started" target="_blank" rel="noopener noreferrer">folium</a> library to display map (using <a href="https://developer.here.com/documentation/map-tile/dev_guide/topics/resource-base-basetile.html" target="_blank" rel="noopener noreferrer">HERE Map Tile API</a>) with markers.

In [ ]:
try:
    import folium
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install folium 
    else:
        !pip install folium 

In [ ]:
import random

load_balance = random.randint(1, 4)  # between 1-4
map_type = 'base'         # aerial, base, traffic
tile_type = 'streettile'  # basetile, maptile, streettile, etc.
map_version = 'newest'
scheme = 'normal.day'
zoom = 13
column = 4400             # any number between 0 and (2**zoom - 1)
row = 2686                # any number between 0 and(2**zoom - 1)
size = 256                # image size (256 or 512)
img_format = 'png8'


# HERE MAP Tile API:
# GET: https://{1-4}.{map_type}.maps.ls.hereapi.com/maptile/2.1/{tile_type}/{map_version}/{scheme}/{zoom}/{column}/{row}/{size}/{img_format}?apiKey={HERE_API_KEY}
tiles_url = 'https://%s.%s.maps.ls.hereapi.com/maptile/2.1/%s/%s/%s/{z}/{x}/{y}/%s/%s?apiKey=%s' % (
    load_balance,
    map_type,
    tile_type,
    map_version,
    scheme,
    size,
    img_format,
    HERE_APIKEY
)

In [ ]:
import folium

here_map = folium.Map(location=[41.878, -87.629],
                      zoom_start=12,
                      tiles=tiles_url,
                      attr='HERE Map')

for library in libraries:
    lt = library.y
    lg = library.x
    folium.Marker([lt, lg]).add_to(here_map)

here_map

#### Get route summaries /distance between all points

We will use the <a href="https://developer.here.com/documentation/routing/dev_guide/topics/resource-calculate-matrix.html" target="_blank" rel="noopener noreferrer">HERE's Matrix API</a> to compute driving distances.

In [ ]:
response_format = 'json'
route_mode = 'shortest;car;traffic:disabled;'
summary_attributes = 'routeId,distance'  # traveltime, costfactor, distance, routeId


# HERE Matrix Routing API:
# POST: https://matrix.route.ls.hereapi.com/routing/7.2/calculatematrix.{response_format}?mode={route_mode}&summaryAttributes={summary_attributes}&apiKey={HERE_API_KEY}
matrix_routing_url = 'https://matrix.route.ls.hereapi.com/routing/7.2/calculatematrix.%s?mode=%s&summaryAttributes=%s&apiKey=%s' % (
    response_format,
    route_mode,
    summary_attributes,
    HERE_APIKEY
)


# Request should not contain more than 15 starts
nb_starts = 10

In [ ]:
import requests

# Ensure unique points
libraries = set(libraries)

waypoints = [b.waypoint for b in libraries]

dest_waypoints = {}
for i, w in enumerate(waypoints):
    dest_waypoints['destination{}'.format(i)] = w



start_coords = [waypoints[i:i+nb_starts] for i in range(0, len(waypoints), nb_starts)]

route_summaries = {}

for sc in start_coords:
    start_waypoints = {}
    for i, s in enumerate(sc):
        start_waypoints['start{}'.format(i)] = s
    
    coords = {**start_waypoints, **dest_waypoints}
    r = requests.post(matrix_routing_url, data = coords)
    json_response = r.json()
    
    for entry in json_response['response']['matrixEntry']:
        key = start_waypoints['start{}'.format(entry['startIndex'])] + '_' + dest_waypoints['destination{}'.format(entry['destinationIndex'])]
        route_summaries[key] = {
            'distance': entry['summary']['distance'],
            'route_id': entry['summary']['routeId']
        }

print('Calculated the distance of {} routes'.format(len(route_summaries)))


In [ ]:
def get_distance(p1, p2):
    return route_summaries[p1.waypoint + '_' + p2.waypoint]['distance']


After running the above code, the data is displayed but it is impossible to determine where to ideally open the coffee shops by just looking at the map.

Let's set up DOcplex to write and solve an optimization model that will help us determine where to locate the coffee shops in an optimal way.

### Step 4: Set up the prescriptive model

In [ ]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

#### Create the DOcplex model
The model contains all the business constraints and defines the objective.

In [ ]:
from docplex.mp.model import Model

mdl = Model("coffee shops")

#### Define the decision variables

In [ ]:
BIGNUM = 99999999

# Ensure unique points
# libraries = set(libraries)
# For simplicity, let's consider that coffee shops candidate locations are the same as libraries locations.
# That is: any library location can also be selected as a coffee shop.
coffeeshop_locations = libraries

# Decision vars
# Binary vars indicating which coffee shop locations will be actually selected
coffeeshop_vars = mdl.binary_var_dict(coffeeshop_locations, name="is_coffeeshop")
#
# Binary vars representing the "assigned" libraries for each coffee shop
link_vars = mdl.binary_var_matrix(coffeeshop_locations, libraries, "link")

#### Express the business constraints
First constraint: if the distance is suspect, it needs to be excluded from the problem.

In [ ]:
for c_loc in coffeeshop_locations:
    for b in libraries:
        if get_distance(c_loc, b) >= BIGNUM:
            mdl.add_constraint(link_vars[c_loc, b] == 0, "ct_forbid_{0!s}_{1!s}".format(c_loc, b))

Second constraint: each library must be linked to a coffee shop that is open.

In [ ]:
mdl.add_constraints(link_vars[c_loc, b] <= coffeeshop_vars[c_loc]
                   for b in libraries
                   for c_loc in coffeeshop_locations)
mdl.print_information()

Third constraint: each library is linked to exactly one coffee shop.

In [ ]:
mdl.add_constraints(mdl.sum(link_vars[c_loc, b] for c_loc in coffeeshop_locations) == 1
                   for b in libraries)
mdl.print_information()

Fourth constraint: there is a fixed number of coffee shops to open.

In [ ]:
# Total nb of open coffee shops
mdl.add_constraint(mdl.sum(coffeeshop_vars[c_loc] for c_loc in coffeeshop_locations) == nb_shops)

# Print model information
mdl.print_information()

#### Express the objective

The objective is to minimize the total distance from libraries to coffee shops so that a book reader always gets to our coffee shop easily.


In [ ]:
# Minimize total distance from points to hubs
total_distance = mdl.sum(link_vars[c_loc, b] * get_distance(c_loc, b) for c_loc in coffeeshop_locations for b in libraries)
mdl.minimize(total_distance)

#### Solve with the Decision Optimization solve service

Solve the model.

In [ ]:
print("# coffee shops locations = %d" % len(coffeeshop_locations))
print("# coffee shops           = %d" % nb_shops)

assert mdl.solve(), "!!! Solve of the model fails"

### Step 6: Investigate the solution and then run an example analysis

The solution can be analyzed by displaying the location of the coffee shops on a map.

In [ ]:
total_distance = mdl.objective_value
open_coffeeshops = [c_loc for c_loc in coffeeshop_locations if coffeeshop_vars[c_loc].solution_value == 1]
not_coffeeshops = [c_loc for c_loc in coffeeshop_locations if c_loc not in open_coffeeshops]
edges = [(c_loc, b) for b in libraries for c_loc in coffeeshop_locations if int(link_vars[c_loc, b]) == 1]

print("Total distance = %g" % total_distance)
print("# coffee shops  = {0}".format(len(open_coffeeshops)))
for c in open_coffeeshops:
    print("new coffee shop: {0!s}".format(c))

In [ ]:
route_mode = 'shortest;car;traffic:disabled;'
route_attributes = 'shape'

# HERE Get Route API
# GET: https://route.ls.hereapi.com/routing/7.2/getroute.{response_format}?mode={route_mode}&routeAttributes={route_attributes}&apiKey={HERE_API_KEY}&routeId={route_id}
get_route_url = 'https://route.ls.hereapi.com/routing/7.2/getroute.json?routeAttributes=%s&mode=%s&apiKey=%s&routeId=' % (
    route_attributes,
    route_mode,
    HERE_APIKEY
)

In [ ]:
route_paths = []

for (c, b) in edges:
    route_id = route_summaries[c.waypoint + '_' + b.waypoint]['route_id']
    route = requests.get(get_route_url + route_id).json()
    
    try:
        route_shape = route['response']['route']['shape']
        route_shapes = []

        for shape in route_shape:
            route_shapes.append(list(map(float, shape.split(','))))
        route_paths.append(route_shapes)
    except:
        print(route)

print('Calculated the shape of {} routes'.format(len(route_shapes)))

#### Displaying the solution
Coffee shops are highlighted in red.

In [ ]:
import folium

here_map = folium.Map(location=[41.878, -87.629],
                      zoom_start=12,
                      tiles=tiles_url,
                      attr='HERE Map')

for coffeeshop in open_coffeeshops:
    lt = coffeeshop.y
    lg = coffeeshop.x
    folium.Marker([lt, lg], icon=folium.Icon(color='red',icon='info-sign')).add_to(here_map)
    
for b in libraries:
    if b not in open_coffeeshops:
        lt = b.y
        lg = b.x
        folium.Marker([lt, lg]).add_to(here_map)

# display the routes
for i, route_path in enumerate(route_paths):
    here_map.add_child(folium.PolyLine(route_path, color='blue', weight=5, opacity=0.5))

here_map

## Summary

You have learned how to set up, formulate and solve an optimization model using Decision Optimization in Watson Studio.

#### References
* <a href="https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html" target="_blank" rel="noopener noreferrer">Decision Optimization CPLEX Modeling for Python documentation</a>
* <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html" target="_blank" rel="noopener noreferrer">Watson Studio documentation</a>

Copyright © 2020-2021 IBM. This notebook and its source code are released under the terms of the MIT License.